In [1]:
import bats
import numpy as np
import matplotlib.pyplot as plt
import time

In [15]:
def freudenthal_grid(m, n):
    """
    Freudenthal triangulation of a m x n grid
    """
    
    def _get_idx(i, j):
        """
        get index of grid in row-major order
        """
        return j + n * i;
    
    
    X = bats.SimplicialComplex()
    
    for i in range(m-1):
        for j in range(n-1):
            k1 = _get_idx(i,j)
            k2 = _get_idx(i+1,j)
            k3 = _get_idx(i,j+1)
            k4 = _get_idx(i+1,j+1)
            X.add_recursive([k1,k2,k4])
            X.add_recursive([k1,k3,k4])

    return X

In [2]:
def freudenthal_grid_light(m, n):
    """
    Freudenthal triangulation of a m x n grid
    """
    
    def _get_idx(i, j):
        """
        get index of grid in row-major order
        """
        return j + n * i;
    
    
    X = bats.LightSimplicialComplex(m*n, 2)
    
    for i in range(m-1):
        for j in range(n-1):
            k1 = _get_idx(i,j)
            k2 = _get_idx(i+1,j)
            k3 = _get_idx(i,j+1)
            k4 = _get_idx(i+1,j+1)
            X.add_recursive([k1,k2,k4])
            X.add_recursive([k1,k3,k4])

    return X

In [6]:
n = 50 
img = np.empty((n,n), dtype=np.float64)
for i in range(n):
    for j in range(n):
        img[i,j] = np.sin(10* np.pi * i / n) + np.cos(10* np.pi * j/n)

img2 = img + 0.01 * np.random.randn(n,n)

m, n = img.shape
X = freudenthal_grid_light(m, n)
t0 = time.monotonic()
vals, imap = bats.lower_star_light_filtration(X, img.flatten())
t1 = time.monotonic()
print("time to extend: {} sec.".format(t1 - t0))

time to extend: 0.0010964309985865839 sec.


In [10]:
F = bats.FilteredLightSimplicialComplex(X, vals)
R = bats.reduce(F, bats.F2())
vals, imap = bats.lower_star_light_filtration(X, img2.flatten())
R.update_filtration(vals)

In [18]:
def time_BATS_update(img, img2):
    m, n = img.shape
    X = freudenthal_grid(m, n)
    
    t0 = time.monotonic()
    vals, imap = bats.lower_star_filtration(X, img.flatten())
    t1 = time.monotonic()
    print("time to extend: {} sec.".format(t1 - t0))
    
    t0 = time.monotonic()
    F = bats.FilteredSimplicialComplex(X, vals)
    t1 = time.monotonic()
    print("time to construct: {} sec.".format(t1 - t0))
    
    t0 = time.monotonic()
    R = bats.reduce(F, bats.F2())
    t1 = time.monotonic()
    print("time to reduce: {} sec.".format(t1 - t0))
    
    t0 = time.monotonic()
    vals, imap = bats.lower_star_filtration(X, img2.flatten())
    R.update_filtration(vals)
    t1 = time.monotonic()
    print("time to update: {} sec.".format(t1 - t0))
    
    t0 = time.monotonic()
    vals, imap = bats.lower_star_filtration(X, img2.flatten())
    F = bats.FilteredSimplicialComplex(X, vals)
    R = bats.reduce(F, bats.F2())
    t1 = time.monotonic()
    print("img2 from scratch: {} sec.".format(t1 - t0))

In [19]:
import gudhi as gd
import time

def time_BATS_update_light(img, img2):
    m, n = img.shape
    X = freudenthal_grid_light(m, n)
    
    t0 = time.monotonic()
    vals, imap = bats.lower_star_light_filtration(X, img.flatten())
    t1 = time.monotonic()
    print("time to extend: {} sec.".format(t1 - t0))
    
    t0 = time.monotonic()
    F = bats.FilteredLightSimplicialComplex(X, vals)
    t1 = time.monotonic()
    print("time to construct: {} sec.".format(t1 - t0))
    
    t0 = time.monotonic()
    R = bats.reduce(F, bats.F2())
    t1 = time.monotonic()
    print("time to reduce: {} sec.".format(t1 - t0))
    
    t0 = time.monotonic()
    vals, imap = bats.lower_star_light_filtration(X, img2.flatten())
    R.update_filtration(vals)
    t1 = time.monotonic()
    print("time to update: {} sec.".format(t1 - t0))
    
    t0 = time.monotonic()
    vals, imap = bats.lower_star_light_filtration(X, img2.flatten())
    F = bats.FilteredLightSimplicialComplex(X, vals)
    R = bats.reduce(F, bats.F2())
    t1 = time.monotonic()
    print("img2 from scratch: {} sec.".format(t1 - t0))


for n in [50,100]:
    print("\n{}".format(n))
    img = np.empty((n,n), dtype=np.float64)
    for i in range(n):
        for j in range(n):
            img[i,j] = np.sin(10* np.pi * i / n) + np.cos(10* np.pi * j/n)
    
    img2 = img + 0.01 * np.random.randn(n,n)
    
    print("SimplicialComplex()")
    time_BATS_update(img, img2)
    print("LightSimplicialComplex()")
    time_BATS_update_light(img, img2)


50
SimplicialComplex()
time to extend: 0.0016063570001279004 sec.
time to construct: 0.0026922699980787 sec.
time to reduce: 0.009897072995954659 sec.
time to update: 0.006485824000264984 sec.
img2 from scratch: 0.016421132000687066 sec.
LightSimplicialComplex()
time to extend: 0.0012882940063718706 sec.
time to construct: 0.0010878840039367788 sec.
time to reduce: 0.009546443005092442 sec.
time to update: 0.00443379699572688 sec.
img2 from scratch: 0.013880204998713452 sec.

100
SimplicialComplex()
time to extend: 0.00517252400459256 sec.
time to construct: 0.010415356999146752 sec.
time to reduce: 0.05999591900035739 sec.
time to update: 0.0220790639941697 sec.
img2 from scratch: 0.09029764799925033 sec.
LightSimplicialComplex()
time to extend: 0.004513628999120556 sec.
time to construct: 0.0037407939962577075 sec.
time to reduce: 0.04401130200130865 sec.
time to update: 0.022229869005968794 sec.
img2 from scratch: 0.07112901200162014 sec.
